In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
import rasterio.mask
import fiona
import json

In [ ]:
from requests import get as requests_get

In [ ]:
from urllib import request

url = 'https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/brazil-states.geojson'
with request.urlopen(url) as f:
    brazil_states = json.load(f)

In [ ]:
FILE_PATH = "generated_files\gbif.parquet"

In [ ]:
# Ocurrence species data -> OSD_df
FILE_PATH = "generated_files\gbif.parquet"
OSD_df = pd.read_parquet(FILE_PATH)

# Creating Geometry
OSD_df['geometry'] = list(zip(OSD_df["Longitude"], OSD_df["Latitude"]))
OSD_df['geometry'] = OSD_df["geometry"].apply(Point)

# Create the geodataframe
OSD_geoframe = gpd.GeoDataFrame(
    OSD_df,
    crs = {'init': 'epsg:4326'},
    geometry = OSD_df['geometry']
)
OSD_geoframe = OSD_geoframe.to_crs("EPSG:4326")
OSD_geoframe.reset_index(drop=True, inplace = True)

OSD_geoframe['Year'] = OSD_geoframe['Data'].dt.year
OSD_df = pd.DataFrame(OSD_geoframe)

In [ ]:
OSD_geoframe.shape

In [ ]:
OSD_geoframe = OSD_geoframe[~OSD_geoframe['Estado'].isnull()]

In [ ]:
coord_list = [(x,y) for x,y in zip(OSD_geoframe['geometry'].x , OSD_geoframe['geometry'].y)]

In [ ]:
# Source: IBGE:  bcim_2016_21_11_2018
INFOS_UFS = gpd.read_file("assets/bcim_2016_21_11_2018.gpkg", layer = 'lim_unidade_federacao_a')[['sigla','geometry']].rename(columns={'sigla':'stateProvince'})
INFOS_UFS.to_file("assets/UFS_JSON", driver = "GeoJSON")
with open("assets/UFS_JSON") as geofile:
    j_file = json.load(geofile)

In [ ]:
img_config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'species_distribution',
    'scale':4 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

In [ ]:
if False:
    fig = px.choropleth_mapbox(
            INFOS_UFS,
            locations="stateProvince",
            center={"lat": -16.95, "lon": -47.78},
            geojson=brazil_states,
            featureidkey='properties.sigla', # add
            color_continuous_scale="Redor",
            opacity=0.,
            zoom=3,
            hover_data={"stateProvince": True},
            mapbox_style="carto-positron"
    )
    fig_graph = px.scatter_mapbox(
        OSD_geoframe.sort_values(by='Year',ascending=True),
        lat="decimalLatitude",
        lon="decimalLongitude",
        hover_name="stateProvince",
        color_discrete_sequence=[px.colors.qualitative.Plotly[1]],
        zoom=3,
        height=500,
        opacity=1
    ).data[0]

    fig.add_trace(fig_graph)
    fig.show(config=img_config)


In [ ]:
fig = px.scatter_mapbox(
    OSD_geoframe.sort_values(by='Year',ascending=True),
    lat="Latitude",
    lon="Longitude",
    hover_name="Estado",
    color_discrete_sequence=[px.colors.qualitative.Plotly[1]],
    zoom=4,
    height=500,
    opacity=1
)
fig.update_layout(autosize=True,margin={"r": 0, "t": 0, "l": 0, "b": 0},mapbox_style="carto-positron")
fig.update_geos(
    visible=False,
    resolution=50,
    lataxis_gridcolor="red",
    lataxis_showgrid=True,
    lataxis_dtick=15,
    lonaxis_showgrid=True,
    lonaxis_dtick=15,
)
#fig.write_image('figure.png', scale=2)
fig.show(config=img_config)

In [ ]:
individual_count_per_state = OSD_geoframe[['Contagem de individuos','Plataforma']].groupby(['Plataforma']).sum().sort_values(by='Contagem de individuos')
fig = px.bar(
    individual_count_per_state,
    x='Contagem de individuos',
    y=individual_count_per_state.index,
    title="Soma de observações por Plataforma",
    orientation='h',text_auto='.1s')
fig.show(config=img_config)

In [ ]:
individual_count_per_state = OSD_geoframe[['Contagem de individuos','Plataforma']].groupby(['Plataforma']).count().sort_values(by='Contagem de individuos')
fig = px.bar(
    individual_count_per_state,
    x='Contagem de individuos',
    y=individual_count_per_state.index,
    title="Soma de ocorrências por Plataforma",
    orientation='h',text_auto='.1s'
)
fig.update_layout(xaxis=dict(title="Observações"), yaxis=dict(title="Plataforma"))
fig.update_xaxes()
fig.show(config=img_config)

In [ ]:
individual_count_per_state = OSD_geoframe[['Contagem de individuos','Estado']].groupby(['Estado']).count().sort_values(by='Contagem de individuos')
fig = px.bar(
    individual_count_per_state,
    x='Contagem de individuos',
    y=individual_count_per_state.index,
    title="Contagem de ocorrências por estado",
    orientation='h',text_auto='.1s'
    
)
fig.update_layout(xaxis=dict(title="Observações"), yaxis=dict(title="Coleção"))
fig.show(config=img_config)

In [ ]:
df = OSD_geoframe.groupby('Year').count()['Contagem de individuos']
fig = px.bar(df, x=df.index, y="Contagem de individuos",text_auto='.1s', title="Observações coletadas por ano")
fig.update_layout(xaxis=dict(title="Ano"), yaxis=dict(title="Observações"))
fig.update_xaxes()
fig.show(config=img_config)

In [ ]:
individual_count_per_state = OSD_geoframe[['Contagem de individuos','Fonte do registro']].groupby(['Fonte do registro']).count().sort_values(by='Contagem de individuos')
fig = px.bar(
    individual_count_per_state,
    x=individual_count_per_state.index,
    y='Contagem de individuos',
    title="Contagem de ocorrências por fonte do registro",
    orientation='v',text_auto=True
    
)
fig.update_layout(xaxis=dict(title="Fonte do registro"), yaxis=dict(title="Contagem de individuos"))
fig.show(config=img_config)

In [ ]:
OSD_geoframe = OSD_geoframe[OSD_geoframe['Year'] > 2000].reset_index(drop=True).copy()
df = OSD_geoframe.groupby('Year').count()['Contagem de individuos']
fig = px.bar(df, x=df.index, y="Contagem de individuos",text_auto='.1s', title="Observações coletadas por ano (Somente do ano de 2000 em diante)")
fig.update_layout(xaxis=dict(title="Ano"), yaxis=dict(title="Observações"))
fig.update_xaxes()
fig.show(config=img_config)